In [1]:
import copy
from gc import set_debug
import heapq as hp
import numpy as np
from queue import PriorityQueue
import time
import math

In [ ]:
def main():
    choice = input("Welcome to cfeng017(861285128)'s 8 puzzle solver\nChoose from the options:\n1: choose from list of default puzzles\n2: enter your own puzzle\n")
    
    choice = int(choice)
    
    
    if choice == 1:
        difficulty = input("\nHow complex should the default puzzle be?\n1: Trivial\n2: Very Easy\n3: Easy\n4: Doable\n5: Oh Boy\n")
        difficulty = int(difficulty)
        puzzle = defaults(difficulty)
        
    elif choice == 2:
        row1 = input("\nEnter elements seperated by spaces\nEnter the first row:  ")
        row2 = input("Enter the second row: ")
        row3 = input("Enter the third row:  ")
        
        puzzle = [row1.split(' '), row2.split(' '), row3.split(' ')]
        for row in range(len(puzzle)):
            for element in range(len(puzzle[row])):
                puzzle[row][element] = int(puzzle[row][element])
                
    heuristic = input("\nEnter your choice of algorithm:\n1: Uniform Cost Search\n2: A* with the Misplaced Tile heuristic\n3: A* with the Euclidean Distance heuristic\n")
    heuristic = int(heuristic)
    
    
    goal = [[1, 2, 3],
            [4, 5, 6],
            [7, 8, 0]]
    # print(puzzle)
    
    search(puzzle, goal, heuristic)
    
def defaults(difficulty):
    if difficulty == 1:
        return [[1, 2, 3],
                [4, 5, 6],
                [7, 8, 0]]
    elif difficulty == 2:
        return [[1, 2, 3],
                [4, 5, 6],
                [7, 0, 8]]
    elif difficulty == 3:
        return [[1, 2, 0],
                [4, 5, 3],
                [7, 8, 6]]
    elif difficulty == 4:
        return [[0, 1, 2],
                [4, 5, 3],
                [7, 8, 6]]
    elif difficulty == 5:
        return [[8, 7, 1],
                [6, 0, 2],
                [5, 4, 3]]
    elif difficulty == 6:
        return [[1, 0, 3],
                [4, 2, 6],
                [7, 5, 8]]
    
    
def search(puzzle, goal, heuristic):
    initial = Node(puzzle, cost(puzzle, goal, heuristic), 0)
    count = 1
    
    #nodes that we still need to explore
    #https://pythonguides.com/priority-queue-in-python/
    frontier = PriorityQueue()
    visited = []
    qsize = 1 
    qmax = -1
    #initialize the frontier using the the initial state of the problem
    frontier.put(initial)
    
    while True:
        #if the frontier is empty then goal cant be found
        if frontier.empty():
            print("Could not find solution")
            break
        qmax = max(qmax, qsize)
        #pop from the frontier and look at next node
        current = frontier.get()
        qsize -= 1
        print('\nThe best state to expand with g(n) = ' + str(current.gn) + ' and h(n) = ' +
              str(current.hn) + ' is...\n' )
        
        #https://stackoverflow.com/questions/17870612/printing-a-two-dimensional-array-in-python
        print(np.matrix(current.puzzle))
        
        # print("currently " + str(count) + " nodes expanded")
        # print("queue max so far: " + str(qmax))
        
        visited.append(current.puzzle)
        #if the current node matchs goal then goal is found
        if current.puzzle == goal:
            print("\nGoal Found!")
            print("To solve this problem the search algorithm expanded a total of " + str(count) + " nodes")
            print("The maximum number of nodes in the queue at any one time: " + str(qmax))
            print("The depth of the goal node was "+ str(current.gn))
            break
            
        print("Expanding this node... ")
        
        #expand on current node and get children
        children = expand(current, goal, heuristic)
        
        # while children:
        #     t = children.pop()
        #     print(np.matrix(t.puzzle))
        
        #loop through the list of children
        while children:
            temp = children.pop()
            #in the node has not been expanded before then add to frontier
            if not temp.puzzle in visited:
                frontier.put(temp)
                count += 1
                qsize += 1
            
        
def expand(current, goal, heuristic):
    children = []
    blank = getblank(current)
    # print(blank)
    
    #http://www.idc-online.com/technical_references/pdfs/information_technology/Pairs_in_Python.pdf
    x, y = blank
    
    #move blank tile up
    if x > 0:
        child = copy.deepcopy(current)
        #swap the tiles
        #https://www.programiz.com/python-programming/examples/swap-variables
        child.puzzle[x][y], child.puzzle[x-1][y] = child.puzzle[x-1][y],child.puzzle[x][y]
        #increase depth
        child.gn += 1
        #calc the heuristic
        child.hn = cost(child.puzzle, goal, heuristic)
        #add child to children list
        children.append(child)
        
    #move blank tile down
    if x < len(current.puzzle)-1:
        child = copy.deepcopy(current)
        
        child.puzzle[x][y], child.puzzle[x+1][y] = child.puzzle[x+1][y],child.puzzle[x][y]
        child.gn += 1
        child.hn = cost(child.puzzle, goal, heuristic)
        
        children.append(child)
    
    #move blank tile left
    if y > 0:
        child = copy.deepcopy(current)
        
        child.puzzle[x][y], child.puzzle[x][y-1] = child.puzzle[x][y-1],child.puzzle[x][y]
        child.gn += 1
        child.hn = cost(child.puzzle, goal, heuristic)
        
        children.append(child)
    
    #move blank tile right
    if y < len(current.puzzle)-1:
        child = copy.deepcopy(current)
        
        child.puzzle[x][y], child.puzzle[x][y+1] = child.puzzle[x][y+1],child.puzzle[x][y]
        child.gn += 1
        child.hn = cost(child.puzzle, goal, heuristic)
        
        children.append(child)
        
    return children


def getblank(current):
    #iterate through the grid
    for i in range(len(current.puzzle)):
        for j in range(len(current.puzzle[i])):
            #if its a 0 then blank is found
            if 0 == current.puzzle[i][j]:
                #take note of the coordinates of the blank tile
                pos = (i, j)
                break 
    return pos
    
def cost(current, goal, heuristic):
    #uniform cost search
    if heuristic == 1:
        return 0;
    #A* with the misplaced tile heiristic
    elif heuristic == 2:
        return misplaced(current,goal);
    #A* with the euclidean distance heuristic
    else:
        return euclidean(current,goal);
    
def misplaced(puzzle, goal):
    count = 0
    
    for i in range(len(puzzle)):
        for j in range(len(puzzle[i])):
            #is the tile is in wrong position and not a blank then add to cost
            if goal[i][j] != puzzle[i][j] and puzzle[i][j] != 0:
                count +=1
    return count

def euclidean(puzzle, goal):
    cost = 0
   
    for tile in range(1, 9):
        a = 0
        b = 0
        x = 0
        y = 0
        for i in range(len(puzzle)):
            for j in range(len(puzzle)):
                if puzzle[i][j] == tile:
                    a = i
                    b = j
                if goal[i][j] == tile:
                    x = i
                    y = j
        dist = math.sqrt(pow(a - x, 2) + pow(b - y, 2))
        cost += dist
    return cost
                
    
class Node:
    def __init__(self, puzzle, hn, gn):
            self.puzzle = puzzle
            self.hn = hn
            self.gn = gn
    def __lt__(self, other):
        return other.hn + other.gn > self.hn + self.gn
            
    
    
    
    
if __name__ == '__main__':
    main()